In [1]:
import pyarrow as pa

from haystack import Document, Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.fastembed import FastembedDocumentEmbedder, FastembedTextEmbedder

from lancedb_haystack import LanceDBDocumentStore, LanceDBEmbeddingRetriever, LanceDBFTSRetriever


/home/eyad/haystack/hastack_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lance_db_path = 'lancedb'
metadata_schema = pa.struct([
    ('author', pa.string()),
    ('title', pa.string()),
    ('url', pa.string())
])
document_store = LanceDBDocumentStore(database=lance_db_path, table_name="documents", metadata_schema=metadata_schema, embedding_dims=384)

pa.field("id", pa.string(), nullable=False),

embedding_retriever = LanceDBEmbeddingRetriever(document_store)

In [3]:
import lancedb
import pyarrow as pa

# connect to LanceDB Cloud
db = lancedb.connect(
    uri=lance_db_path
)

# create an empty table with schema
data = [
    {"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
    {"vector": [5.9, 26.5], "item": "bar", "price": 20.0},
    {"vector": [10.2, 100.8], "item": "baz", "price": 30.0},
    {"vector": [1.4, 9.5], "item": "fred", "price": 40.0},
]

schema = pa.schema([
    pa.field("vector", pa.list_(pa.float32(), 2)),
    pa.field("item", pa.utf8()),
    pa.field("price", pa.float32()),
])

table_name = "basic_ingestion_example"
table = db.create_table(table_name, schema=schema, mode="overwrite")
# Add data
table.add(data)

AddResult(version=4)

In [5]:
db = lancedb.connect("play")
schema = pa.schema(
        [
            pa.field("id", pa.string(), nullable=False),
            pa.field("vector", pa.list_(pa.float32(), list_size=300)),
            pa.field("content", pa.string()),
            pa.field("dataframe", pa.string()),  # Using a string, so we can jam the dataframe in as json.
            pa.field("blob", pa.binary()),
        ]
    )
table = db.create_table(name="ho", schema=schema, on_bad_vectors="fill", fill_value=0)
table

LanceTable(name='ho', version=1, _conn=LanceDBConnection(uri='/mnt/c/Users/Me/PycharmProjects/Data-Science/haystack/Document Stores/LanceDB/play'))

In [17]:
from haystack import Document
d = Document("asa")

print(d.to_dict(flatten=False))

{'id': 'asa', 'content': None, 'blob': None, 'meta': {}, 'score': None, 'embedding': None, 'sparse_embedding': None}


In [ ]:
column_data = {}
for field in schema:
    values = []
    for doc in doc_dicts:
        val = doc.get(field.name)
        values.append(val)
    column_data[field.name] = values

new_data = pa.table(column_data, schema=schema)

In [6]:
import pyarrow as pa

from haystack import Document, Pipeline
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.embedders.fastembed import FastembedDocumentEmbedder, FastembedTextEmbedder

from lancedb_haystack import LanceDBDocumentStore, LanceDBEmbeddingRetriever, LanceDBFTSRetriever

In [7]:
lance_db_path = 'lancedb'
metadata_schema = pa.struct([
    ('author', pa.string()),
    ('title', pa.string()),
    ('url', pa.string())
])
document_store = LanceDBDocumentStore(database=lance_db_path, table_name="documents", metadata_schema=metadata_schema, embedding_dims=384)


embedding_retriever = LanceDBEmbeddingRetriever(document_store)

In [8]:

embedder = FastembedDocumentEmbedder()
embedder.warm_up()

Fetching 5 files: 100%|██████████| 5/5 [02:58<00:00, 35.80s/it] 


In [9]:

p = Pipeline()
p.add_component(instance=DocumentCleaner(), name="cleaner")
p.add_component(instance=DocumentSplitter(split_by="sentence", split_length=250, split_overlap=30), name="splitter")
p.add_component(instance=embedder, name="embedder")
p.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
p.connect("cleaner.documents", "splitter.documents")
p.connect("splitter.documents", "embedder.documents")
p.connect("embedder.documents", "writer.documents")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - embedder: FastembedDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (list[Document])
  - splitter.documents -> embedder.documents (list[Document])
  - embedder.documents -> writer.documents (List[Document])

In [10]:
docs = [
    Document(
        content="A man must appear somewhat vain, who declares that he has been obliged to reject much useful information, for fear of increasing too much the size of his work: and yet manages to find room for a few pages of his own, by way of Preface: but lest the objects which the compiler of this little work has had in view should be mistaken, he finds it absolutely necessary to say a few words in explanation of them. This small collection of military memorandums was originally intended only for the compiler’s own pocket; to assist him in the execution of his duty: but it occurred to him, that many of his military friends stood in equal need of such an aid, and would willingly give a few shillings for what they would not be at the trouble of collecting. The compiler has seen young men, on their first entry into the regiment of artillery, give a guinea for manuscripts, which contained a very small part of the information offered in this little book.",
        meta={
            'title': 'The Bombardier, and Pocket Gunner.',
            'author':'Ralph Willet Adye',
            'url':'https://gutenberg.org/cache/epub/73283/pg73283-images.html'
        }
    ),
    Document(
        content="26th July, 1799.—Left H. H. this morning to make a tour into Wales and the west of England. Ld. H. went to dine at St. Anne’s, and Drew, Charles, and myself came to this place (General Fitzpatrick’s, Sunninghill) to meet him. On Saturday, ye 27th, we arrived to dinner at Oxford. Close by the General’s, upon Ascot Heath, there is a camp. The cavalry were watered whilst we went on the road, a pretty incident; their brilliant coats contrasted with the dusky brown of the heath, and produced a very lively effect. A cross road to Henley; just as we entered the town we passed under Park Place, a late purchase of Ld. Malmesbury’s. Somewhere near Henley is the highest ground south of Trent. On ye 28th went from Oxford, by Chapel House, to Warwick. I could not help laughing at the event of our going, because the subject of seeing the Castle without being obliged by urgent solicitations to make a long visit there (as Ld. H. was sure we should be very much pressed) had occupied his thoughts for many hours. Ld. H. had written letters to Ly. W. and Mrs. Smith, so calculated as to make a refusal of the pressing invitations (which he dreaded having) quite civil; but we need not have been embarrassed. So far from being solicited to stay, the whole family flew to their fortresses in some battlemented towers, gave out[2] that they were walking, and shunned us. It was a most fortunate escape to me, tho’ I believe Ld. H. was vexed at the impertinence of the proceeding. The Castle is magnificent, decorated with ornaments appropriate to the building, and in the very best taste possible. His taste in virtu is better than in breeding, as he manifested a gross deficiency of that quality to us.",
        meta={
            'title': 'The journal of Elizabeth Lady Holland',
            'author':'Elizabeth Lady Holland',
            'url':'https://gutenberg.org/cache/epub/73280/pg73280-images.html'
        }
    )
]

In [11]:
result = p.run({"cleaner": {"documents": docs}})

Calculating embeddings: 100%|██████████| 2/2 [00:00<00:00,  6.75it/s]


PipelineRuntimeError: The following component failed to run:
Component name: 'writer'
Component type: 'DocumentWriter'
Error: lance error: Append with different schema: fields in different order, expected: [id, vector, content, dataframe, blob, meta, _isempty], actual: [vector, meta, blob, id, content, dataframe, _isempty], `id` should have nullable=false but nullable=true, location: /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/lance-core-0.39.0/src/datatypes/schema.rs:200:27

In [ ]:
result